# MODEL LLM


In [128]:
from vertexai.generative_models import (
    GenerativeModel,
    GenerationConfig,
    HarmCategory,
    HarmBlockThreshold
)

model = GenerativeModel('gemini-1.5-flash-002')
generation_config = GenerationConfig(
    temperature=1,  # Example temperature value
    # top_k=1,         # Example top-k value
    # top_p=0.1        # Example top-p value
)

def generate_content(prompt):
    result = model.generate_content(
        contents=prompt,
        safety_settings={
            HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
        },
        generation_config=generation_config
    )
    return result

# FUNCTION

In [129]:
import pandas as pd
from io import StringIO
from google.cloud import storage
import os
BASE_DIR = os.getcwd()

def load_transcription_csv(bucket_name, file_path):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(file_path)
    content = blob.download_as_text()

    return pd.read_csv(StringIO(content))[['speaker', 'transcript']]

def get_prompt(**kwargs):
    with open('prompt/disposition_code.txt', 'r') as file:
        prompt_template = file.read()

    return prompt_template.format(**kwargs)

def get_ref_table():
    excel_path = "disposition_code_ref_table.xlsx"
    ref_table_df = pd.read_excel(os.path.join(BASE_DIR, excel_path))
    ref_table_df.drop(columns=["Konkatenasi_Layer123"], inplace=True)
    ref_table_dict = ref_table_df.to_dict(orient='records')
    return ref_table_dict

def generate_prompt(bucket_name, file_path, interaction_id: str):
    transcript_df = load_transcription_csv(bucket_name,f"{file_path}/transcription/{interaction_id}/transcription_results.csv")
    transcript_df = transcript_df.where(pd.notnull(transcript_df), 'silence')
    json_dict = {
        'speaker': transcript_df['speaker'].tolist(),
        'transcript': transcript_df['transcript'].tolist()
    }
    prompt = get_prompt(
        transcript=json_dict,
        disposition_code_ref_table=get_ref_table()
    )
    return prompt

# BATCH PROCESS

In [130]:
import json

list_of_unknown_csv = [9151919416110000090, 9151919416110000091, 9151919416110000092]
output_data = []
for i in range(9151919416110000051, 9151919416110000099+1):
    if i in list_of_unknown_csv:
        continue
    try:
        prompt = generate_prompt('athena-nonprod-gcs', 'athena-ws4a/athena-qapm', i)
        response = generate_content(prompt).text
        disposition_result = json.loads(response.replace("```json", "").replace("```", "").strip())
        output_data.append({
            'interaction_id': i,
            'option_1': disposition_result[0]['value'],
            'explanation_1': disposition_result[0]['field3'],
            'option_2': disposition_result[1]['value'] if len(disposition_result) > 1 else None,
            'explanation_2': disposition_result[1]['field3'] if len(disposition_result) > 1 else None,
            'option_3': disposition_result[2]['value'] if len(disposition_result) > 2 else None,
            'explanation_3': disposition_result[2]['field3'] if len(disposition_result) > 2 else None,
        })
        print(f"Processed interaction_id: {i}")
    except Exception as e:
        print(f"Error: {e} at interaction_id: {i}")

df = pd.DataFrame(output_data)


Processed interaction_id: 9151919416110000051
Processed interaction_id: 9151919416110000052
Processed interaction_id: 9151919416110000053
Processed interaction_id: 9151919416110000054
Processed interaction_id: 9151919416110000055
Processed interaction_id: 9151919416110000056
Processed interaction_id: 9151919416110000057
Processed interaction_id: 9151919416110000058
Processed interaction_id: 9151919416110000059
Processed interaction_id: 9151919416110000060
Processed interaction_id: 9151919416110000061
Processed interaction_id: 9151919416110000062
Processed interaction_id: 9151919416110000063
Processed interaction_id: 9151919416110000064
Processed interaction_id: 9151919416110000065
Processed interaction_id: 9151919416110000066
Processed interaction_id: 9151919416110000067
Processed interaction_id: 9151919416110000068
Processed interaction_id: 9151919416110000069
Processed interaction_id: 9151919416110000070
Processed interaction_id: 9151919416110000071
Processed interaction_id: 91519194

In [131]:
df

,interaction_id,option_1,explanation_1,option_2,explanation_2,option_3,explanation_3
0,9151919416110000051,Complain (General Customer)|Credit Card|CRD: S...,Penjelasan klasifikasi: Nasabah mengalami gaga...,Report Status Checking|Credit Card|CRD: Credit...,Penjelasan klasifikasi: Laporan gagal tarik tu...,Request (General Customer)|Credit Card|CRD: Cr...,Penjelasan klasifikasi: Nasabah meminta formul...
1,9151919416110000052,Complain (General Customer)|Credit Card|CRD: C...,Penjelasan klasifikasi: Nasabah melaporkan tra...,Request (General Customer)|Credit Card|CRD: Cr...,Penjelasan klasifikasi: Nasabah meminta pemblo...,Request (General Customer)|Credit Card|CRD: Cr...,Penjelasan klasifikasi: Nasabah meminta pencet...
2,9151919416110000053,Complain (General Customer)|Branchless|BLB: OC...,Penjelasan klasifikasi: Nasabah mengalami kend...,Inquiry (General Customer)|Branchless|BLB: ATM...,Penjelasan klasifikasi: Nasabah menanyakan ke...,Request (General Customer)|Branchless|BLB: OCT...,Penjelasan klasifikasi: Permintaan laporan ter...
3,9151919416110000054,Complain (General Customer)|Credit Card|CRD: C...,Penjelasan klasifikasi: Nasabah menanyakan tag...,Complain (General Customer)|Credit Card|CRD: C...,Penjelasan klasifikasi: Agen membutuhkan bant...,Request (General Customer)|Credit Card|CRD: Cr...,Penjelasan klasifikasi: Supervisor menginstruk...
4,9151919416110000055,Complain (General Customer)|Branchless|BLB: OC...,Penjelasan klasifikasi: Nasabah mengalami kend...,Complain (General Customer)|Credit Card|CRD: C...,Penjelasan klasifikasi: Nasabah komplain menge...,Complain (General Customer)|Credit Card|CRD: P...,Penjelasan klasifikasi: Nasabah menanyakan det...
5,9151919416110000056,Inquiry (General Customer)|Funding|FND: Tabung...,Penjelasan klasifikasi: Nasabah menanyakan bia...,Request (General Customer)|Branchless|BLB: Kar...,Penjelasan klasifikasi: Nasabah menanyakan car...,Inquiry (General Customer)|Branchless|BLB: OCT...,Penjelasan klasifikasi: Nasabah menanyakan per...
6,9151919416110000057,Inquiry (General Customer)|Lending|LND: KPR Ot...,Penjelasan klasifikasi: Nasabah menanyakan sis...,Request (General Customer)|Lending|LND: KPR Ot...,Penjelasan klasifikasi: Nasabah meminta inform...,None,None
7,9151919416110000058,Inquiry (General Customer)|Credit Card|CRD: Limit,Penjelasan klasifikasi: Nasabah menanyakan sis...,Inquiry (General Customer)|Credit Card|CRD: Limit,Penjelasan klasifikasi: Nasabah menanyakan dam...,Inquiry (General Customer)|Credit Card|CRD: Limit,Penjelasan klasifikasi: Nasabah ingin mengetah...
8,9151919416110000059,Complain (General Customer)|Credit Card|CRD: P...,Penjelasan klasifikasi: Nasabah menanyakan men...,Request (General Customer)|Credit Card|CRD: Cr...,Penjelasan klasifikasi: Nasabah meminta kenaik...,Complain (General Customer)|Credit Card|CRD: C...,Penjelasan klasifikasi: Nasabah menanyakan men...
9,9151919416110000060,Complain (General Customer)|Credit Card|CRD: C...,Penjelasan klasifikasi: Nasabah menanyakan ter...,Inquiry (General Customer)|Credit Card|CRD: Cr...,Penjelasan klasifikasi: Nasabah ingin mengetah...,Request (General Customer)|Credit Card|CRD: Cr...,Penjelasan klasifikasi: Verifikasi data nasaba...


# Combine data with ground truth

In [132]:
# read json file
import json

with open('./ground_truth_dirty/ground_truth.json') as f:
    ground_truth = json.load(f)

# map with ground truth
df['interaction_id'] = df['interaction_id'].astype(str)
df['ground_truth'] = df['interaction_id'].map(ground_truth)

# save to csv
df.to_csv('output.csv', index=False)

# Read the output.csv file

In [133]:
df = pd.read_csv('output.csv')
df

,interaction_id,option_1,explanation_1,option_2,explanation_2,option_3,explanation_3,ground_truth
0,9151919416110000051,Complain (General Customer)|Credit Card|CRD: S...,Penjelasan klasifikasi: Nasabah mengalami gaga...,Report Status Checking|Credit Card|CRD: Credit...,Penjelasan klasifikasi: Laporan gagal tarik tu...,Request (General Customer)|Credit Card|CRD: Cr...,Penjelasan klasifikasi: Nasabah meminta formul...,Complain (General Customer)|Branchless|BLB: AT...
1,9151919416110000052,Complain (General Customer)|Credit Card|CRD: C...,Penjelasan klasifikasi: Nasabah melaporkan tra...,Request (General Customer)|Credit Card|CRD: Cr...,Penjelasan klasifikasi: Nasabah meminta pemblo...,Request (General Customer)|Credit Card|CRD: Cr...,Penjelasan klasifikasi: Nasabah meminta pencet...,Complain (General Customer)|Credit Card|CRD: C...
2,9151919416110000053,Complain (General Customer)|Branchless|BLB: OC...,Penjelasan klasifikasi: Nasabah mengalami kend...,Inquiry (General Customer)|Branchless|BLB: ATM...,Penjelasan klasifikasi: Nasabah menanyakan ke...,Request (General Customer)|Branchless|BLB: OCT...,Penjelasan klasifikasi: Permintaan laporan ter...,Complain (General Customer)|Branchless|BLB: OC...
3,9151919416110000054,Complain (General Customer)|Credit Card|CRD: C...,Penjelasan klasifikasi: Nasabah menanyakan tag...,Complain (General Customer)|Credit Card|CRD: C...,Penjelasan klasifikasi: Agen membutuhkan bant...,Request (General Customer)|Credit Card|CRD: Cr...,Penjelasan klasifikasi: Supervisor menginstruk...,Complain (General Customer)|Credit Card|CRD: C...
4,9151919416110000055,Complain (General Customer)|Branchless|BLB: OC...,Penjelasan klasifikasi: Nasabah mengalami kend...,Complain (General Customer)|Credit Card|CRD: C...,Penjelasan klasifikasi: Nasabah komplain menge...,Complain (General Customer)|Credit Card|CRD: P...,Penjelasan klasifikasi: Nasabah menanyakan det...,Complain (General Customer)|Branchless|BLB: OC...
5,9151919416110000056,Inquiry (General Customer)|Funding|FND: Tabung...,Penjelasan klasifikasi: Nasabah menanyakan bia...,Request (General Customer)|Branchless|BLB: Kar...,Penjelasan klasifikasi: Nasabah menanyakan car...,Inquiry (General Customer)|Branchless|BLB: OCT...,Penjelasan klasifikasi: Nasabah menanyakan per...,Inquiry (General Customer)|Funding (FND)|FND: ...
6,9151919416110000057,Inquiry (General Customer)|Lending|LND: KPR Ot...,Penjelasan klasifikasi: Nasabah menanyakan sis...,Request (General Customer)|Lending|LND: KPR Ot...,Penjelasan klasifikasi: Nasabah meminta inform...,NaN,NaN,Inquiry (General Customer)|Lending|LND: KPR Ot...
7,9151919416110000058,Inquiry (General Customer)|Credit Card|CRD: Limit,Penjelasan klasifikasi: Nasabah menanyakan sis...,Inquiry (General Customer)|Credit Card|CRD: Limit,Penjelasan klasifikasi: Nasabah menanyakan dam...,Inquiry (General Customer)|Credit Card|CRD: Limit,Penjelasan klasifikasi: Nasabah ingin mengetah...,Inquiry (General Customer)|Credit Card|CRD: Cr...
8,9151919416110000059,Complain (General Customer)|Credit Card|CRD: P...,Penjelasan klasifikasi: Nasabah menanyakan men...,Request (General Customer)|Credit Card|CRD: Cr...,Penjelasan klasifikasi: Nasabah meminta kenaik...,Complain (General Customer)|Credit Card|CRD: C...,Penjelasan klasifikasi: Nasabah menanyakan men...,Inquiry (General Customer)|Credit Card|CRD: Pe...
9,9151919416110000060,Complain (General Customer)|Credit Card|CRD: C...,Penjelasan klasifikasi: Nasabah menanyakan ter...,Inquiry (General Customer)|Credit Card|CRD: Cr...,Penjelasan klasifikasi: Nasabah ingin mengetah...,Request (General Customer)|Credit Card|CRD: Cr...,Penjelasan klasifikasi: Verifikasi data nasaba...,Feedback/Report (General Customer)|Credit Card...


# CALCULATE ACCURACY

In [134]:
import pandas as pd

# check whether option_1 or option_2 or option_3 is equal to ground_truth, the result is True or False in "is_match" column
df['is_match'] = (df['ground_truth'] == df['option_1']) | (df['ground_truth'] == df['option_2']) | (df['ground_truth'] == df['option_3'])
df_view = df[['interaction_id', 'option_1', 'option_2', 'option_3', 'ground_truth', 'is_match']]
df_view


,interaction_id,option_1,option_2,option_3,ground_truth,is_match
0,9151919416110000051,Complain (General Customer)|Credit Card|CRD: S...,Report Status Checking|Credit Card|CRD: Credit...,Request (General Customer)|Credit Card|CRD: Cr...,Complain (General Customer)|Branchless|BLB: AT...,False
1,9151919416110000052,Complain (General Customer)|Credit Card|CRD: C...,Request (General Customer)|Credit Card|CRD: Cr...,Request (General Customer)|Credit Card|CRD: Cr...,Complain (General Customer)|Credit Card|CRD: C...,True
2,9151919416110000053,Complain (General Customer)|Branchless|BLB: OC...,Inquiry (General Customer)|Branchless|BLB: ATM...,Request (General Customer)|Branchless|BLB: OCT...,Complain (General Customer)|Branchless|BLB: OC...,False
3,9151919416110000054,Complain (General Customer)|Credit Card|CRD: C...,Complain (General Customer)|Credit Card|CRD: C...,Request (General Customer)|Credit Card|CRD: Cr...,Complain (General Customer)|Credit Card|CRD: C...,False
4,9151919416110000055,Complain (General Customer)|Branchless|BLB: OC...,Complain (General Customer)|Credit Card|CRD: C...,Complain (General Customer)|Credit Card|CRD: P...,Complain (General Customer)|Branchless|BLB: OC...,True
5,9151919416110000056,Inquiry (General Customer)|Funding|FND: Tabung...,Request (General Customer)|Branchless|BLB: Kar...,Inquiry (General Customer)|Branchless|BLB: OCT...,Inquiry (General Customer)|Funding (FND)|FND: ...,False
6,9151919416110000057,Inquiry (General Customer)|Lending|LND: KPR Ot...,Request (General Customer)|Lending|LND: KPR Ot...,NaN,Inquiry (General Customer)|Lending|LND: KPR Ot...,True
7,9151919416110000058,Inquiry (General Customer)|Credit Card|CRD: Limit,Inquiry (General Customer)|Credit Card|CRD: Limit,Inquiry (General Customer)|Credit Card|CRD: Limit,Inquiry (General Customer)|Credit Card|CRD: Cr...,False
8,9151919416110000059,Complain (General Customer)|Credit Card|CRD: P...,Request (General Customer)|Credit Card|CRD: Cr...,Complain (General Customer)|Credit Card|CRD: C...,Inquiry (General Customer)|Credit Card|CRD: Pe...,False
9,9151919416110000060,Complain (General Customer)|Credit Card|CRD: C...,Inquiry (General Customer)|Credit Card|CRD: Cr...,Request (General Customer)|Credit Card|CRD: Cr...,Feedback/Report (General Customer)|Credit Card...,False


In [135]:
# calculate the mean of "is_match" column where ground_truth is not NaN
accuracy = df[df['ground_truth'].notnull()]['is_match'].mean()
accuracy*100

np.float64(34.146341463414636)

In [136]:
# GET WHICH LAYER GET THE WRONG PREDICTION
def split_option(df, column_name):
    """Memisahkan kolom berdasarkan '|' dan membuat kolom baru."""
    df[column_name + '_layer_1'] = df[column_name].str.split('|').str[0]
    df[column_name + '_layer_2'] = df[column_name].str.split('|').str[1]
    df[column_name + '_layer_3'] = df[column_name].str.split('|').str[2]
    return df

df = split_option(df, 'option_1')
df = split_option(df, 'option_2')
df = split_option(df, 'option_3')
df = split_option(df, 'ground_truth')

# check if layer 1, layer 2, layer 3 match with ground_truth layer 1, layer 2, layer 3
df['is_match_layer_1'] = (df['ground_truth_layer_1'] == df['option_1_layer_1']) | (df['ground_truth_layer_1'] == df['option_2_layer_1']) | (df['ground_truth_layer_1'] == df['option_3_layer_1'])
df['is_match_layer_2'] = (df['ground_truth_layer_2'] == df['option_1_layer_2']) | (df['ground_truth_layer_2'] == df['option_2_layer_2']) | (df['ground_truth_layer_2'] == df['option_3_layer_2'])
df['is_match_layer_3'] = (df['ground_truth_layer_3'] == df['option_1_layer_3']) | (df['ground_truth_layer_3'] == df['option_2_layer_3']) | (df['ground_truth_layer_3'] == df['option_3_layer_3'])

In [137]:
df

,interaction_id,option_1,explanation_1,option_2,explanation_2,option_3,explanation_3,ground_truth,is_match,option_1_layer_1,...,option_2_layer_3,option_3_layer_1,option_3_layer_2,option_3_layer_3,ground_truth_layer_1,ground_truth_layer_2,ground_truth_layer_3,is_match_layer_1,is_match_layer_2,is_match_layer_3
0,9151919416110000051,Complain (General Customer)|Credit Card|CRD: S...,Penjelasan klasifikasi: Nasabah mengalami gaga...,Report Status Checking|Credit Card|CRD: Credit...,Penjelasan klasifikasi: Laporan gagal tarik tu...,Request (General Customer)|Credit Card|CRD: Cr...,Penjelasan klasifikasi: Nasabah meminta formul...,Complain (General Customer)|Branchless|BLB: AT...,False,Complain (General Customer),...,CRD: Credit Cards Transaksi,Request (General Customer),Credit Card,CRD: Credit Cards Form Sanggahan Transaksi,Complain (General Customer),Branchless,BLB: ATM Gagal Tarik,True,False,False
1,9151919416110000052,Complain (General Customer)|Credit Card|CRD: C...,Penjelasan klasifikasi: Nasabah melaporkan tra...,Request (General Customer)|Credit Card|CRD: Cr...,Penjelasan klasifikasi: Nasabah meminta pemblo...,Request (General Customer)|Credit Card|CRD: Cr...,Penjelasan klasifikasi: Nasabah meminta pencet...,Complain (General Customer)|Credit Card|CRD: C...,True,Complain (General Customer),...,CRD: Credit Cards Blokir/Buka Blokir,Request (General Customer),Credit Card,CRD: Credit Cards Kirim Ulang Kartu,Complain (General Customer),Credit Card,CRD: Credit Cards Sanggahan Transaksi,True,True,True
2,9151919416110000053,Complain (General Customer)|Branchless|BLB: OC...,Penjelasan klasifikasi: Nasabah mengalami kend...,Inquiry (General Customer)|Branchless|BLB: ATM...,Penjelasan klasifikasi: Nasabah menanyakan ke...,Request (General Customer)|Branchless|BLB: OCT...,Penjelasan klasifikasi: Permintaan laporan ter...,Complain (General Customer)|Branchless|BLB: OC...,False,Complain (General Customer),...,BLB: ATM Others,Request (General Customer),Branchless,BLB: OCTO Pay Others,Complain (General Customer),Branchless,BLB: OCTO Mobile Kendala Transaksi,True,True,False
3,9151919416110000054,Complain (General Customer)|Credit Card|CRD: C...,Penjelasan klasifikasi: Nasabah menanyakan tag...,Complain (General Customer)|Credit Card|CRD: C...,Penjelasan klasifikasi: Agen membutuhkan bant...,Request (General Customer)|Credit Card|CRD: Cr...,Penjelasan klasifikasi: Supervisor menginstruk...,Complain (General Customer)|Credit Card|CRD: C...,False,Complain (General Customer),...,CRD: Credit Cards Transaksi,Request (General Customer),Credit Card,CRD: Credit Cards Transaksi,Complain (General Customer),Credit Card,CRD: Credit Cards Sanggahan Transaksi,True,True,False
4,9151919416110000055,Complain (General Customer)|Branchless|BLB: OC...,Penjelasan klasifikasi: Nasabah mengalami kend...,Complain (General Customer)|Credit Card|CRD: C...,Penjelasan klasifikasi: Nasabah komplain menge...,Complain (General Customer)|Credit Card|CRD: P...,Penjelasan klasifikasi: Nasabah menanyakan det...,Complain (General Customer)|Branchless|BLB: OC...,True,Complain (General Customer),...,CRD: Credit Cards Sanggahan Transaksi,Complain (General Customer),Credit Card,CRD: Pembayaran Tagihan Kartu Kredit,Complain (General Customer),Branchless,BLB: OCTO Mobile Kendala Login,True,True,True
5,9151919416110000056,Inquiry (General Customer)|Funding|FND: Tabung...,Penjelasan klasifikasi: Nasabah menanyakan bia...,Request (General Customer)|Branchless|BLB: Kar...,Penjelasan klasifikasi: Nasabah menanyakan car...,Inquiry (General Customer)|Branchless|BLB: OCT...,Penjelasan klasifikasi: Nasabah menanyakan per...,Inquiry (General Customer)|Funding (FND)|FND: ...,False,Inquiry (General Customer),...,BLB: Kartu Debit Others,Inquiry (General Customer),Branchless,BLB: OCTO Mobile Kendala Transaksi,Inquiry (General Customer),Funding (FND),FND: Tabungan Others,True,False,False
6,9151919416110000057,Inquiry (General Customer)|Lending|LND: KPR Ot...,Penjelasan klasifikasi: Nasabah menanyaka

In [138]:
accuracy_layer_1 = df[df['ground_truth_layer_1'].notnull()]['is_match_layer_1'].mean()
print(f"Layer 1 accuracy: {accuracy_layer_1}")
# show df for layer 1
# df[['interaction_id', 'option_1_layer_1','option_2_layer_1','option_3_layer_1', 'ground_truth_layer_1', 'is_match_layer_1']]
df[['interaction_id', 'option_1_layer_1', 'ground_truth_layer_1', 'is_match_layer_1']]

Layer 1 accuracy: 0.9024390243902439


,interaction_id,option_1_layer_1,ground_truth_layer_1,is_match_layer_1
0,9151919416110000051,Complain (General Customer),Complain (General Customer),True
1,9151919416110000052,Complain (General Customer),Complain (General Customer),True
2,9151919416110000053,Complain (General Customer),Complain (General Customer),True
3,9151919416110000054,Complain (General Customer),Complain (General Customer),True
4,9151919416110000055,Complain (General Customer),Complain (General Customer),True
5,9151919416110000056,Inquiry (General Customer),Inquiry (General Customer),True
6,9151919416110000057,Inquiry (General Customer),Inquiry (General Customer),True
7,9151919416110000058,Inquiry (General Customer),Inquiry (General Customer),True
8,9151919416110000059,Complain (General Customer),Inquiry (General Customer),False
9,9151919416110000060,Complain (General Customer),Feedback/Report (General Customer),False


In [139]:
accuracy_layer_2 = df[df['ground_truth_layer_2'].notnull()]['is_match_layer_2'].mean()
print(f"Layer 2 accuracy: {accuracy_layer_2}")
df[['interaction_id','option_1_layer_2','option_2_layer_2','option_3_layer_2', 'ground_truth_layer_2', 'is_match_layer_2']]

Layer 2 accuracy: 0.926829268292683


,interaction_id,option_1_layer_2,option_2_layer_2,option_3_layer_2,ground_truth_layer_2,is_match_layer_2
0,9151919416110000051,Credit Card,Credit Card,Credit Card,Branchless,False
1,9151919416110000052,Credit Card,Credit Card,Credit Card,Credit Card,True
2,9151919416110000053,Branchless,Branchless,Branchless,Branchless,True
3,9151919416110000054,Credit Card,Credit Card,Credit Card,Credit Card,True
4,9151919416110000055,Branchless,Credit Card,Credit Card,Branchless,True
5,9151919416110000056,Funding,Branchless,Branchless,Funding (FND),False
6,9151919416110000057,Lending,Lending,NaN,Lending,True
7,9151919416110000058,Credit Card,Credit Card,Credit Card,Credit Card,True
8,9151919416110000059,Credit Card,Credit Card,Credit Card,Credit Card,True
9,9151919416110000060,Credit Card,Credit Card,Credit Card,Credit Card,True


In [140]:
accuracy_layer_3 = df[df['ground_truth_layer_3'].notnull()]['is_match_layer_3'].mean()
print(f"Layer 3 accuracy: {accuracy_layer_3}")
df[['interaction_id','option_1_layer_3','option_2_layer_3','option_3_layer_3', 'ground_truth_layer_3', 'is_match_layer_3']]

Layer 3 accuracy: 0.4634146341463415


,interaction_id,option_1_layer_3,option_2_layer_3,option_3_layer_3,ground_truth_layer_3,is_match_layer_3
0,9151919416110000051,CRD: Sanggahan Transaksi,CRD: Credit Cards Transaksi,CRD: Credit Cards Form Sanggahan Transaksi,BLB: ATM Gagal Tarik,False
1,9151919416110000052,CRD: Credit Cards Sanggahan Transaksi,CRD: Credit Cards Blokir/Buka Blokir,CRD: Credit Cards Kirim Ulang Kartu,CRD: Credit Cards Sanggahan Transaksi,True
2,9151919416110000053,BLB: OCTO Pay Kendala Transaksi,BLB: ATM Others,BLB: OCTO Pay Others,BLB: OCTO Mobile Kendala Transaksi,False
3,9151919416110000054,CRD: Credit Cards Tagihan,CRD: Credit Cards Transaksi,CRD: Credit Cards Transaksi,CRD: Credit Cards Sanggahan Transaksi,False
4,9151919416110000055,BLB: OCTO Mobile Kendala Login,CRD: Credit Cards Sanggahan Transaksi,CRD: Pembayaran Tagihan Kartu Kredit,BLB: OCTO Mobile Kendala Login,True
5,9151919416110000056,FND: Tabungan Biaya/ Denda,BLB: Kartu Debit Others,BLB: OCTO Mobile Kendala Transaksi,FND: Tabungan Others,False
6,9151919416110000057,LND: KPR Others,LND: KPR Others,NaN,LND: KPR Others,True
7,9151919416110000058,CRD: Limit,CRD: Limit,CRD: Limit,CRD: Credit Cards Limit,False
8,9151919416110000059,CRD: Pembayaran Tagihan Kartu Kredit,CRD: Credit Cards Naik/Turun Limit Sementara,CRD: Credit Cards Transaksi,CRD: Pembayaran Tagihan Kartu Kredit,True
9,9151919416110000060,CRD: Credit Cards Annual Fee,CRD: Credit Cards Annual Fee,CRD: Credit Cards Annual Fee,CRD: Credit Cards,False
